In [1]:
from config.Experiment_Config import ExperimentConfig
from exp.trainer import Trainer


c:\Users\yunkai\.conda\envs\Multi_turn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = ExperimentConfig.get_default_config()
config.tokenizer_settings.max_length = 512
config.training_settings.batch_size = 32
config.data_settings.imbalanced_strategy = 'weighted_sampler'
config.data_settings.weighted_sampler_alpha = 0.5
config.model_settings.weight_init = 'kaiming_normal'
config.model_settings.init_hidden_dim = 384
config.model_settings.activation = 'gelu'
config.model_settings.fine_tune_embedding = False
config.training_settings.early_stopping_patience = 20
config.training_settings.gradient_clip = None

In [3]:
trainer = Trainer(
    config=config
)

# Train model
metrics_history = trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


✅ Created all necessary directories

EXPERIMENT CONFIGURATION

📊 Experiment Name: BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse
📁 Model Directory: saved_models\BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse
📈 Results Directory: results\BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse

🔧 Model Configuration:
- Model Type: BiLSTM
- Embedding Type: BERT_base_uncased
- Hidden Dimensions: [256, 128]
- Bidirectional: True
- Dropout Rate: 0.1
- Attention: False
- Fine-tune Embedding: False
- Weight Init: kaiming_normal
- Activation: gelu

⚙️ Training Configuration:
- Batch Size: 32
- Max Length: 512
- Learning Rate: 0.001
- Weight Decay: 0.01
- Num Epochs: 100
- Loss Function: cross_entropy
- Optimizer: adam
- Early Stopping Patience: 20
- Scheduler Patience: 3
- Scheduler Factor: 0.1
- Min Learning Rate: 1e-06
- Checkpoint Frequency: 10

📊 Data Configuration:
- Imbalanced Strategy: w

Training: 100%|██████████| 127/127 [01:18<00:00,  1.61it/s]
c:\Users\yunkai\.conda\envs\Multi_turn\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Evaluating (val): 100%|██████████| 15/15 [00:14<00:00,  1.06it/s]
c:\Users\yunkai\.conda\envs\Multi_turn\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Epoch 1 Results:

Training Metrics:
accuracy: 0.1257
precision_macro: 0.0043
precision_micro: 0.1257
recall_macro: 0.0259
recall_micro: 0.1257
f1_macro: 0.0064
f1_micro: 0.1257
loss: 3.5690

Validation Metrics:
accuracy: 0.4958
precision_macro: 0.0130
precision_micro: 0.4958
recall_macro: 0.0263
recall_micro: 0.4958
f1_macro: 0.0174
f1_micro: 0.4958
loss: 3.2138
❌ Error saving confusion matrix
Attempted path: results\BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse\figures\latest\train_confusion_matrix_BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse.png
Error details: [Errno 2] No such file or directory: 'C:\\Users\\yunkai\\OneDrive\\Program\\Py\\project\\Multi_turn\\results\\BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse\\figures\\latest\\train_confusion_matrix_BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse.png'


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\yunkai\\OneDrive\\Program\\Py\\project\\Multi_turn\\results\\BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse\\figures\\latest\\train_confusion_matrix_BiLSTM_BERT_base_uncased_512_2_cross_entropy_kaiming_normal_weighted_sampler_AFalse.png'